# Testing ML
> Quick launch into Variables, Functions, Arrays, Classes, HTML.

- layout: default

In [1]:
import os
import certifi
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths to your dataset and tags
dataset_path = '/Users/poonam/Desktop/imgdata'
tags = ['animals', 'food', 'people']  # Replace with your own tags

# Define the parameters for training the model
image_size = (224, 224)
batch_size = 32
num_epochs = 10

# Create data generators for training and validation
data_generator = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)
train_generator = data_generator.flow_from_directory(dataset_path, target_size=image_size, batch_size=batch_size, subset='training')
validation_generator = data_generator.flow_from_directory(dataset_path, target_size=image_size, batch_size=batch_size, subset='validation')

# Set the SSL certificate file environment variable
os.environ['SSL_CERT_FILE'] = certifi.where()

# Load the pre-trained VGG16 model without the top classification layer
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(image_size[0], image_size[1], 3))

# Add a global average pooling layer and a dense layer for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(tags), activation='softmax')(x)

# Create the model with the VGG16 base and the classification layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_generator, epochs=num_epochs, validation_data=validation_generator)

# Save the trained model
model.save('image_tagging_model.h5')



2023-05-16 09:38:20.994806: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Found 10 images belonging to 3 classes.
Found 0 images belonging to 3 classes.
Epoch 1/10


2023-05-16 09:38:30.395749: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1/1 [==============================] - 6s 6s/step - loss: 1.0756 - accuracy: 0.6000
Epoch 2/10
1/1 [==============================] - 5s 5s/step - loss: 0.8388 - accuracy: 0.8000
Epoch 3/10
1/1 [==============================] - 5s 5s/step - loss: 0.6416 - accuracy: 0.8000
Epoch 4/10
1/1 [==============================] - 5s 5s/step - loss: 0.4779 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 6s 6s/step - loss: 0.3786 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 5s 5s/step - loss: 0.2976 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 5s 5s/step - loss: 0.2223 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 5s 5s/step - loss: 0.1663 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 5s 5s/step - loss: 0.1286 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 5s 5s/step - loss: 0.1013 - accuracy: 1.0000


In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('image_tagging_model.h5')

# Load and preprocess the input image
image_path = '/Users/poonam/Downloads/dwayne.jpeg'  # Replace with the path to your input image
person = Image.open(image_path)
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = img / 255.0  # Normalize the image

# Make predictions
predictions = model.predict(img)
predicted_class = np.argmax(predictions[0])  # Get the index of the highest probability class

# Print the predicted class and corresponding tag
tags = ['animals', 'food', 'people']  # Replace with your own tags
predicted_tag = tags[predicted_class]
print("Image")
person.show()
print("Predicted class:", predicted_class)
print("Predicted tag:", predicted_tag)

1/1 [==============================] - 1s 902ms/step
Image
Predicted class: 2
Predicted tag: people
